# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
#this had to be added to run on my machine
#pip install pyproj

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

import cartopy.crs as ccrs
import geoviews as gv # noqa
import pyproj
    
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,midland,31.9974,-102.0779,279.85,54,0,4.63,US,1669910471
1,1,haines junction,60.7522,-137.5108,260.66,87,100,1.05,CA,1669910723
2,2,rikitea,-23.1203,-134.9692,296.40,73,71,4.86,PF,1669910723
3,3,barrow,71.2906,-156.7887,266.16,93,100,3.60,US,1669910723
4,4,bambous virieux,-20.3428,57.7575,297.71,72,13,3.52,MU,1669910723


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot_4

#Configure the map plot_3

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
)

# Display the map plot
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

narrow_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness","Lat","Lng"])

max_temp = (narrow_df["Max Temp"] <= 300) & (narrow_df["Max Temp"] > 294)
wind_speed = narrow_df["Wind Speed"] < 4
cloudiness = narrow_df["Cloudiness"] == 0

# Display data
newnarrow_df = narrow_df[max_temp & wind_speed & cloudiness]
newnarrow_df


,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng
19,susangerd,297.66,1.29,0,31.5608,48.1831
29,nanakuli,296.54,3.09,0,21.3906,-158.1547
46,salalah,297.31,1.98,0,17.0151,54.0924
174,wahiawa,294.74,3.09,0,21.5028,-158.0236
304,veraval,297.43,2.06,0,20.9000,70.3667
350,una,295.95,2.22,0,20.8167,71.0333
381,pisco,295.18,3.60,0,-13.7000,-76.2167
417,mount isa,297.02,3.60,0,-20.7333,139.5000
539,tahta,294.06,3.43,0,26.7693,31.5021


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = newnarrow_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''

# YOUR CODE HERE
hotel_df

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Hotel Name
19,susangerd,297.66,1.29,0,31.5608,48.1831,
29,nanakuli,296.54,3.09,0,21.3906,-158.1547,
46,salalah,297.31,1.98,0,17.0151,54.0924,
174,wahiawa,294.74,3.09,0,21.5028,-158.0236,
304,veraval,297.43,2.06,0,20.9000,70.3667,
350,una,295.95,2.22,0,20.8167,71.0333,
381,pisco,295.18,3.60,0,-13.7000,-76.2167,
417,mount isa,297.02,3.60,0,-20.7333,139.5000,
539,tahta,294.06,3.43,0,26.7693,31.5021,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
  
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
susangerd - nearest hotel: No hotel found
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
salalah - nearest hotel: Muscat International Hotel
wahiawa - nearest hotel: The Inn At Schofield
veraval - nearest hotel: Shyam
una - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
mount isa - nearest hotel: Ibis Styles
tahta - nearest hotel: No hotel found


,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Hotel Name
19,susangerd,297.66,1.29,0,31.5608,48.1831,No hotel found
29,nanakuli,296.54,3.09,0,21.3906,-158.1547,"Aulani, A Disney Resort & Spa"
46,salalah,297.31,1.98,0,17.0151,54.0924,Muscat International Hotel
174,wahiawa,294.74,3.09,0,21.5028,-158.0236,The Inn At Schofield
304,veraval,297.43,2.06,0,20.9000,70.3667,Shyam
350,una,295.95,2.22,0,20.8167,71.0333,No hotel found
381,pisco,295.18,3.60,0,-13.7000,-76.2167,La Portada
417,mount isa,297.02,3.60,0,-20.7333,139.5000,Ibis Styles
539,tahta,294.06,3.43,0,26.7693,31.5021,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Max Temp",
    scale = 1,
    color = "Hotel Name",
    tiles = "OSM",
    label = "Wind Speed",
    frame_width = 700,
    frame_height = 500,
)

#----- Hotel names obtained by moving the curser over the location-------


# Display the map plot
map_plot_2


:Overlay
   .Tiles.I           :Tiles   [x,y]
   .Points.Wind_Speed :Points   [Lng,Lat]   (Hotel Name,Max Temp)